# Description

This notebook is used to train a RNN on the known universe of SMILES to learn to very accurately generate novel small molecules. We then use this initial network to generate our generation 0 (gen0) candidate molecules.

## Train the Network

In [16]:
import tensorflow
tensorflow.test.is_gpu_available()

True

In [17]:
import numpy as np
from copy import copy

import keras

from lstm_chem.utils.config import process_config
from lstm_chem.model import LSTMChem
from lstm_chem.generator import LSTMChemGenerator
from lstm_chem.trainer import LSTMChemTrainer
from lstm_chem.data_loader import DataLoader

In [18]:
CONFIG_FILE = 'experiments/2019-12-23/LSTM_Chem/config.json'
config = process_config(CONFIG_FILE)

In [19]:
modeler = LSTMChem(config, session='train')

In [6]:
train_dl = DataLoader(config, data_type='train')

loading SMILES...
done.
tokenizing SMILES...


100%|██████████| 438552/438552 [02:08<00:00, 3404.02it/s]

done.


In [7]:
valid_dl = copy(train_dl)
valid_dl.data_type = 'valid'

In [8]:
trainer = LSTMChemTrainer(modeler, train_dl, valid_dl)

In [9]:
trainer.train()

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 771 steps, validate for 86 steps
Epoch 1/42
770/771 [============================>.] - ETA: 0s - loss: 1.3086WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00001: saving model to experiments\2019-12-23\LSTM_Chem\checkpoints/LSTM_Chem-01-0.83.hdf5
771/771 [==============================] - 232s 301ms/step - loss: 1.3082 - val_loss: 0.8268
Epoch 2/42
770/771 [============================>.] - ETA: 0s - loss: 0.9699WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00002: saving model to experiments\2019-12-23\LSTM_Chem\checkpoints/LSTM_Chem-02-0.71.hdf5
771/771 [==============================] - 227s 295ms/st


Epoch 00013: saving model to experiments\2019-12-23\LSTM_Chem\checkpoints/LSTM_Chem-13-0.51.hdf5
771/771 [==============================] - 229s 297ms/step - loss: 0.7305 - val_loss: 0.5144
Epoch 14/42
770/771 [============================>.] - ETA: 0s - loss: 0.7251WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00014: saving model to experiments\2019-12-23\LSTM_Chem\checkpoints/LSTM_Chem-14-0.51.hdf5
771/771 [==============================] - 232s 301ms/step - loss: 0.7251 - val_loss: 0.5090
Epoch 15/42
531/771 [===================>..........] - ETA: 1:09 - loss: 0.7205

KeyError: 'val_loss'

In [ ]:
# Save weights of the trained model
trainer.model.save_weights('experiments/2019-12-23/LSTM_Chem/checkpoints/LSTM_Chem-baseline-model-full.hdf5.hdf5')

## Now load the model and GENERATE new molecules

In [10]:
config['model_weight_filename'] = 'experiments/2019-12-23/LSTM_Chem/checkpoints/LSTM_Chem-baseline-model-full.hdf5'
print(config)

batch_size: 512
checkpoint_dir: experiments\2019-12-23\LSTM_Chem\checkpoints/
checkpoint_mode: min
checkpoint_monitor: val_loss
checkpoint_save_best_only: false
checkpoint_save_weights_only: true
checkpoint_verbose: 1
config_file: experiments/2019-12-23/LSTM_Chem/config.json
data_filename: ./datasets/all_smiles_clean.smi
data_length: 0
exp_dir: experiments\2019-12-23\LSTM_Chem
exp_name: LSTM_Chem
finetune_batch_size: 1
finetune_data_filename: ./datasets/TRPM8_inhibitors_for_fine-tune.smi
finetune_epochs: 5
model_arch_filename: experiments\2019-12-23\LSTM_Chem\model_arch.json
model_weight_filename: experiments/2019-12-23/LSTM_Chem/checkpoints/LSTM_Chem-baseline-model-full.hdf5
num_epochs: 42
optimizer: adam
sampling_temp: 0.75
seed: 71
smiles_max_length: 128
tensorboard_log_dir: experiments\2019-12-23\LSTM_Chem\logs/
tensorboard_write_graph: true
train_smi_max_len: 74
units: 256
validation_split: 0.1
verbose_training: true



In [11]:
modeler = LSTMChem(config, session='generate')
generator = LSTMChemGenerator(modeler)
print(config)

Loading model architecture from experiments\2019-12-23\LSTM_Chem\model_arch.json ...
Loading model checkpoint from experiments/2019-12-23/LSTM_Chem/checkpoints/LSTM_Chem-baseline-model-full.hdf5 ...
Loaded the Model.
batch_size: 512
checkpoint_dir: experiments\2019-12-23\LSTM_Chem\checkpoints/
checkpoint_mode: min
checkpoint_monitor: val_loss
checkpoint_save_best_only: false
checkpoint_save_weights_only: true
checkpoint_verbose: 1
config_file: experiments/2019-12-23/LSTM_Chem/config.json
data_filename: ./datasets/all_smiles_clean.smi
data_length: 0
exp_dir: experiments\2019-12-23\LSTM_Chem
exp_name: LSTM_Chem
finetune_batch_size: 1
finetune_data_filename: ./datasets/TRPM8_inhibitors_for_fine-tune.smi
finetune_epochs: 5
model_arch_filename: experiments\2019-12-23\LSTM_Chem\model_arch.json
model_weight_filename: experiments/2019-12-23/LSTM_Chem/checkpoints/LSTM_Chem-baseline-model-full.hdf5
num_epochs: 42
optimizer: adam
sampling_temp: 0.75
seed: 71
smiles_max_length: 128
tensorboard_log

In [12]:
sample_number = 10000
sampled_smiles = generator.sample(num=sample_number)

100%|██████████| 10000/10000 [50:08<00:00,  3.32it/s] 


In [13]:
from rdkit import RDLogger, Chem, DataStructs
from rdkit.Chem import AllChem, Draw, Descriptors
from rdkit.Chem.Draw import IPythonConsole
RDLogger.DisableLog('rdApp.*')

In [14]:
valid_mols = []
for smi in sampled_smiles:
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        valid_mols.append(mol)
# low validity
print('Validity: ', f'{len(valid_mols) / sample_number:.2%}')

valid_smiles = [Chem.MolToSmiles(mol) for mol in valid_mols]
# high uniqueness
print('Uniqueness: ', f'{len(set(valid_smiles)) / len(valid_smiles):.2%}')

# Of valid smiles generated, how many are truly original vs ocurring in the training data
import pandas as pd
training_data = pd.read_csv('./datasets/all_smiles_clean.smi', header=None)
training_set = set(list(training_data[0]))
original = []
for smile in valid_smiles:
    if not smile in training_set:
        original.append(smile)
print('Originality: ', f'{len(set(original)) / len(set(valid_smiles)):.2%}')

Validity:  94.50%
Uniqueness:  99.20%
Originality:  99.86%


In [15]:
with open('./generations/gen0.smi', 'w') as f:
    for item in valid_smiles:
        f.write("%s\n" % item)